In [2]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np
from tqdm import tqdm
from pprint import pprint

from voltagebudget.util import read_modes
from voltagebudget.util import get_mode_names
from voltagebudget.util import read_stim
from voltagebudget.util import find_E
from voltagebudget.util import find_phis
from voltagebudget.util import create_times
from voltagebudget.util import score_by_n
from voltagebudget.util import mad

from voltagebudget.neurons import adex

def plot_voltages(times, x, step=1e-3, tspan=(0.2, 0.24), random=0, plot_mean=False):
    neurons = list(range(N))
    if random > 0:
        np.random.shuffle(neurons)
        neurons = neurons[0:random]

    p = figure(plot_width=600, plot_height=300)
    for n in neurons:
        p.line(x=times, y=x[:, n], color="black", alpha=0.4)

    if plot_mean:
        p.line(x=times, y=x.mean(1), color='red', alpha=0.8)
    
    p.xaxis.axis_label = 'Time (s)'
    p.yaxis.axis_label = 'V (volts)'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.x_range = Range1d(*tspan)
    show(p)


Loading BokehJS ...

# Shared parameters

In [3]:
# Modes?
get_mode_names()

[u'delayed_regular_bursting',
 u'adaption',
 u'heterogenous',
 u'regular_burst',
 u'regular',
 u'transient_spiking',
 u'delayed_accelerating',
 u'initial_burst']

In [4]:
N = 10
t = 1.25
f = 0
E_0 = 0.2
d = -2e-3

mode = 'heterogenous'
time_step = 1e-5
seed_value = None

params, w_in, bias_in, _ = read_modes(mode)
pprint(params)

{u'C': [1e-10, 2e-10],
 u'V_l': [-0.058, -0.07],
 u'V_rheo': [-0.046, -0.058],
 u'V_t': -0.05,
 u'a': [-1.1e-08, 4e-09],
 u'b': [0.0, 1.2e-10],
 u'delta_t': 0.002,
 u'g_l': [1e-08, 1.8e-08],
 u'tau_w': [0.03, 0.3]}


# Def square stimulus

In [5]:
# Create impulse
stim = (.25e-9, 0.2, 1)

# Reference run

In [ ]:
ns_ref, ts_ref, voltages_ref = adex(
    N,
    t,
    np.array([]),
    np.array([]),
    w_in=w_in,
    bias_in=bias_in/5,
    f=f,
    A=0,
    phi=0,
    sigma=0,
    budget=True,
    save_args=None,
    time_step=time_step,
    pulse_params=stim,
    seed_value=seed_value,
    **params)

In [ ]:
# %debug

In [ ]:
print(ns_ref.size)
print(voltages_ref.keys())

if ns_ref.size > 0:
    E = find_E(E_0, ns_ref, ts_ref, no_lock=False, verbose=False)
else: 
    E = E_0

In [ ]:
times = voltages_ref["times"]
plot_voltages(times, voltages_ref['V_m'].T, tspan=(0, t), random=5)

In [ ]:
times = voltages_ref['times']
v_c = voltages_ref['V_comp'][0, :]
v_o = voltages_ref['V_osc'][0, :] - voltages_ref['V_osc'][0, :]

vs = np.vstack([times, -70e-3 - v_o, -70e-3 - v_c - v_o, -70e-3 - v_c - v_o]).T
header = "t,osc,comp,vm"
np.savetxt("../analysis/vm_hetero_timecourse.csv", 
           vs,
           delimiter=",", 
           header=header,
           comments="")